# Test training code for S2AE

In [2]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import open3d as o3d
from scipy import spatial


import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from tqdm.auto import tqdm

from average_meter import AverageMeter
from data_splitter import DataSplitter
from loss import L2Loss
from model import Model
from sphere import Sphere
from visualize import Visualize
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Initialize some parameter

In [3]:
print(f"Initializing CUDA...")
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True

print(f"Setting parameters...")
bandwidth = 100
n_features = 2
learning_rate = 4.5e-3
n_epochs = 55
batch_size = 13
num_workers = 32
descriptor_size = 256
net_input_size = 2 * bandwidth

print(f"Initializing data structures...")
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
net = Model(n_features,bandwidth).cuda()
criterion = ImprovedTripletLoss(margin=2, alpha=0.5, margin2=0.2)
writer = SummaryWriter()
model_save = 'test_training_params.pkl'

Initializing CUDA...


RuntimeError: cuda runtime error (999) : unknown error at /pytorch/aten/src/THC/THCGeneral.cpp:47

## Load the dataset

In [6]:
export_ds = '/mnt/data/datasets/nuscenes/processed'
img_filename = f"{export_ds}/images.npy"
cloud_filename = f"{export_ds}/clouds.npy"

print(f"Loading from images from {img_filename} and clouds from {cloud_filename}")
img_features = np.load(img_filename)
cloud_features = np.load(cloud_filename)
print(f"Shape of images is {img_features.shape} and clouds is {cloud_features.shape}")

Loading from images from /mnt/data/datasets/nuscenes/processed/images.npy and clouds from /mnt/data/datasets/nuscenes/processed/clouds.npy
Shape of images is (10, 2, 200, 200) and clouds is (10, 2, 200, 200)
